In [155]:
# Run the example from documentation https://pyphi.readthedocs.io/en/nonbinary/examples/multivalued_elements.html

import pyphi
import numpy as np
pyphi.config.PARTITION_TYPE = 'ALL'
pyphi.config.MEASURE = 'AID'
pyphi.config.USE_SMALL_PHI_DIFFERENCE_FOR_CES_DISTANCE = True
pyphi.config.ASSUME_CUTS_CANNOT_CREATE_NEW_CONCEPTS = True

tpm = np.array([
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
])
num_states_per_node = [3, 2, 2]
names = ['P', 'C', 'N']
cm = [
     [0, 1, 1],
     [0, 0, 1],
     [1, 0, 0],
]
network = pyphi.Network(
tpm,
cm=cm,
node_labels=names,
num_states_per_node=num_states_per_node,
)

df = network.tpmdf

state = (0, 0, 1)
subsystem = pyphi.Subsystem(network, state)
sia = pyphi.compute.sia(subsystem)
sia.phi

Computing concepts:   0%|          | 0/7 [00:00<?, ?it/s](12, 12)


0.43872200000000006

In [36]:
# Get the TPMs for 143-168. First import the data
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import random

times = np.squeeze(np.load("Cori_2016-12-14/spikes.times.npy"))
clusters = np.squeeze(np.load("Cori_2016-12-14/spikes.clusters.npy"))
probe = np.squeeze(np.load("Cori_2016-12-14/clusters.probes.npy"))

minindex = min(clusters)
maxindex = max(clusters)

# split data into individual neuron arrays
individual_times = []
for i in range(minindex, maxindex+1):
    indices = np.where(clusters==i)[0]
    individual_times.append(times[indices].astype(float))

# get only the good neurons
annotations = np.squeeze(np.load("Cori_2016-12-14/clusters._phy_annotation.npy"))
good_indices = np.where(annotations >= 2)
good_neurons = np.array(individual_times)[good_indices].tolist()

probe1_indices = np.nonzero(probe)[0]
good_indices_probe1 = np.intersect1d(good_indices,probe1_indices)
print(good_indices_probe1.shape)
good_neurons_probe1 = np.array(individual_times)[good_indices_probe1]

n_143 = good_neurons_probe1[143]
n_168 = good_neurons_probe1[168]

(371,)
/Users/Marcel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/Users/Marcel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [144]:
import math
def binarise_spiketrain(a,S):
    """
    Binarises a spike-train with bins of size S. 
    - a is a list of times when the neuron fired. 
    """
    last = math.ceil(max(a)/S)
    a_states = np.zeros(math.ceil(max(a)/S))
    for spike_t in a:
        index = int(spike_t / S)
        a_states[index] = 1
    return a_states

def get_TPM_index(state):  # rename this
    """
    Given the state of a set of neurons, get the
    index into the TPM that the state corresponds to.

    Example: 
        state = [[1,0,0]
                 [0,1,1]]
        Neuron A is in state 100 and B is in state 011
        State A corresponds to the decimal 4
        State B corresponds to the decimal 3
        The index of A, the first listed state, varies 'fastest' in the TPM according to the PyPhi convention, 
        https://pyphi.readthedocs.io/en/latest/conventions.html.
        So, the indices, as tuples, will be (A0,B0),(A1,B0),(A2,B0),(A3,B0),(A4,B0),...,(A0,B1),(A1,B1)... 
        To convert these tuples to ints, we need A + B*8 = 4 + 3*8 = 28

        In general, where each node has n possible states, this becomes:
            - A*n^0 + B*n^1 + C*n^2 + ... where A, B, C are the states of each node,
            in order from first to last in the state input. 
            - Note that for the binary arrays accepted by this function, 
            n = 2^(m), where m is the size of the binary array that describes the state of a node. 
    """
    n = 2**(state.shape[1])
    index = 0
    for i in range(state.shape[0]):
        dec_node = int("".join(str(int(j)) for j in state[i,:]), 2)
        index += dec_node * n ** i
    return index

def get_TPM_nonbinary(binaryneurons, K, skipby):
    """Given an array of binarised neuron spike-trains
    and a K value for how many time-steps to include in a single state, 
    get the TPM of the system. 
        - Skipby controls where the future state starts: given that the current state
        starts at T, future state starts at T+skipby
    Example: 
        if K = 3, then find the TPM that describes 
        the transition probability of System[t-2,t-1,t] --> System[t+1,t+2,t+3]

    Returns:
        - A TPM of the system in state-state mode (TODO: conventions?)
        - A matrix A with the same dimensions of TPM, where A[i,j] is the 
        number of transitions that were used to calculate the value of TPM[i,j].
    
    """
    assert K >= 1
    assert binaryneurons.shape[0] >= 1

    size = (2**K)**binaryneurons.shape[0]
    
    # initialise TPM and num_transitions arrays
    TPM = np.zeros((size, size))
    num_transitions = np.zeros((size, size))

    # start at K-1 because our state at time i looks BACK to i-1, i-2,.. to build the rest of state
    i = K - 1
    while i + skipby < binaryneurons.shape[1]:
        curr_state = binaryneurons[:,i-(K-1):(i+1)]
        i_c = get_TPM_index(curr_state)

        future_state = binaryneurons[:,i-(K-1) + skipby:(i+1) + skipby]   # Ugly indexing 
        i_f = get_TPM_index(future_state)

        num_transitions[i_c, i_f] += 1
        i += 1
    for j in range(num_transitions.shape[0]):
        total = sum(num_transitions[j,:])
        if total == 0:
            raise ValueError("State with index " + str(j) + " was not observed in the data")
        
        TPM[j,:] = num_transitions[j,:] / total
    
    return TPM, num_transitions

def TPM_from_spiketrains(spiketrains, S, K, skip):

    # get the binarised spike trains for each neuron
    # create a multidimensional array of the spiketrains by not considering further than the shortest train
    binarised_trains = [[] for _ in range(len(spiketrains))]
    min_length = math.inf
    for i in range(len(spiketrains)):
        binarised_trains[i] = binarise_spiketrain(spiketrains[i], S)
        min_length = min(min_length, len(binarised_trains[i]))
    
    binarised_trains = np.array([binarised_trains[i][0:min_length] for i in range(len(binarised_trains))])   # probably slow? 
    # compute the TPM 
    return get_TPM_nonbinary(binarised_trains, K, skip)
     


In [ ]:
def coarse_grain_nonbinary_TPM(TPM, grouping):
    """
    Turns a nonbinary TPM into a binary TPM by coarse-graining each nonbinary element
    according to a grouping.
        - grouping only groups states of individual elements.
        - grouping example for a TPM of 2 elements each w 4 states:
        [[[0, 1], [2, 3]], [[0], [1, 2], [3]]]
            - The first element's 0 and 1 states are grouped into a state, and 2,3 into another.
            - The second element's 0 state stays the same, 1,2 are grouped and 3 stays the same.

    For example, for a TPM of 2 elements, each with 4 states,
    the input TPM will have 4*4 = 16 states. We can coarse grain
    such that only the first three states of each element will be grouped into an OFF state,
    and the last element will be grouped into ON. 

    However, we don't have to coarsegrain to binary. We might say that the first state
    of each element will map to OFF, the second and third will map to FIRING, 
    fourth to BURSTING. 
    """
    



In [164]:

TPM = TPM_from_spiketrains([n_143,n_168],0.005,1,1)
TPM_macro = TPM_from_spiketrains([n_143,n_168],0.005,2,2)


(2, 540489)
(4, 4)
(2, 540489)
(16, 16)
(array([[0.84267563, 0.02638963, 0.02375933, 0.00498561, 0.03313727,
        0.00813027, 0.00518677, 0.00250045, 0.03063682, 0.00452983,
        0.00656176, 0.00192244, 0.00469534, 0.00184351, 0.00161943,
        0.00142592],
       [0.43578209, 0.09726034, 0.07989563, 0.02681182, 0.06316074,
        0.03216519, 0.0236178 , 0.0157452 , 0.09186198, 0.03032075,
        0.0331099 , 0.01641999, 0.01655495, 0.01304602, 0.01079671,
        0.0134509 ],
       [0.48623   , 0.08903982, 0.10332155, 0.03130815, 0.05089319,
        0.02521399, 0.02535355, 0.01446781, 0.06898958, 0.02232974,
        0.03386677, 0.01400261, 0.01186267, 0.00646632, 0.00874581,
        0.00790845],
       [0.27949351, 0.10948116, 0.09002471, 0.05281038, 0.05497221,
        0.03829524, 0.035979  , 0.03165534, 0.08662755, 0.0435454 ,
        0.0501853 , 0.03536133, 0.01791229, 0.02130945, 0.02254478,
        0.02980235],
       [0.52229051, 0.06726108, 0.07306094, 0.01800554, 0.0

In [167]:
import pyphi
import numpy as np
pyphi.config.PARTITION_TYPE = 'ALL'
pyphi.config.MEASURE = 'AID'
pyphi.config.USE_SMALL_PHI_DIFFERENCE_FOR_CES_DISTANCE = True
pyphi.config.ASSUME_CUTS_CANNOT_CREATE_NEW_CONCEPTS = True

num_states_per_node = [4,4]

names = ['A','B']
network = pyphi.Network(
TPM_macro[0],
node_labels=names,
num_states_per_node=num_states_per_node,
)

df = network.tpmdf

state = (0, 0)
subsystem = pyphi.Subsystem(network, state)
sia = pyphi.compute.sia(subsystem)
sia.phi

0.0

In [166]:
TPM_macro[0].shape

(16, 16)

In [172]:
[[
           [0,1],
           [2,3]],
         [ [0],
           [1,2],
           [3]]
         ]

[[[0, 1], [2, 3]], [[0], [1, 2], [3]]]